# Installations

In order to access the YOLO model, we need to first install Ultralytics.

In [2]:
%pip install ultralytics
%pip install opencv-python
%pip install sklearn
%pip install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not wr

# Image Registration (optional)

In [18]:
import os
import cv2
import shutil
from pathlib import Path

# Paths
root_dir = Path("/Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/WiSARD_dataset/registered")
dataset_splits = ["train", "val", "test"]

# Image registration function (thermal aligned to RGB)
def register_thermal_to_rgb(rgb_path, thermal_path, output_path):
    rgb_img = cv2.imread(str(rgb_path), cv2.IMREAD_COLOR)
    thermal_img = cv2.imread(str(thermal_path), cv2.IMREAD_GRAYSCALE)

    # Resize thermal to match RGB if needed
    thermal_resized = cv2.resize(thermal_img, (rgb_img.shape[1], rgb_img.shape[0]))

    # Save aligned thermal image
    output_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(output_path), thermal_resized)

# Copy labels
for split in dataset_splits:
    rgb_dir = root_dir / split / "rgb"
    thermal_dir = root_dir / split / "thermal"
    labels_dir = root_dir / split / "labels"

    # Create output labels dir
    labels_dir.mkdir(parents=True, exist_ok=True)

    for rgb_img_path in rgb_dir.glob("*.jpg"):
        base_name = rgb_img_path.stem

        # 1. Register thermal to RGB
        thermal_img_path = thermal_dir / f"{base_name}.jpg"
        if thermal_img_path.exists():
            register_thermal_to_rgb(rgb_img_path, thermal_img_path, thermal_img_path)

        # 2. Copy label file from source labels folder
        source_label = Path(f"data/{split}/labels/{base_name}.txt")  # adjust if your original labels are elsewhere
        dest_label = labels_dir / f"{base_name}.txt"
        if source_label.exists():
            shutil.copy(source_label, dest_label)

print("Thermal registration and label copying complete.")

Thermal registration and label copying complete.


# Model Training

From Ultralytics, now we can import the YOLO model.

In [3]:
from ultralytics import YOLO
import torch
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, RandomizedSearchCV

/Users/ikmalbasirun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [20]:
print(torch.backends.mps.is_available())  # Should print: True

True


YOLO has different types of model, so we need to specify which version we would like to load. In this case, we will be using a **pre-trained** yolo-version-11.

In [4]:
# Load a model
modelv5 = YOLO("yolov5n.yaml").load("yolov5nu.pt")  # build from YAML and transfer weights
modelv8 = YOLO("yolov8n.yaml").load("yolov8n.pt")  # build from YAML and transfer weights
modelv11 = YOLO("yolo11n.pt") # build from YAML and transfer weights

#using MPS instead of CPU
device = "mps" if torch.backends.mps.is_available() else "cpu"

Transferred 427/427 items from pretrained weights
Transferred 355/355 items from pretrained weights


We're training our model via a YAML file. We would have used a csv file, but this model does not recognise this. YAML files are unfortunately more complicated, so we will discuss this further during the meeting.

# YOLOv5

## Hyperparameter Tuning

In [6]:
# Define search space
search_space = {
    "lr0": (1e-5, 1e-2),
    "momentum": (0.85, 0.98),
    "weight_decay": (1e-4, 1e-2),
    "scale": (0.3, 0.7),
    "hsv_s": (0.3, 0.8)
}

# Tune hyperparameters on WISARD DATASET
modelv5.tune(
    data="wisard_dataset_rgb.yaml",
    epochs=10,
    iterations=10,
    imgsz=416,
    batch=16,
    optimizer="Adam",
    space=search_space,
    device="cpu"
)

Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune16'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/10 with hyperparameters: {'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'scale': 0.5, 'hsv_s': 0.7}


/Users/ikmalbasirun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Transferred 427/427 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.124 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 CPU (Apple M4 Pro)
engine/trainer: task=detect, mode=train, model=yolov5n.yaml, data=wisard_dataset_rgb.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train74, exist_ok=False, pretrained=yolov5nu.pt, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

train: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/rgb/labels.cache... 1128 images, 115 backgrounds, 0 corrupt: 100%|██████████| 1128/1128 [00:00<?, ?it/s]
val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/labels.cache... 375 images, 59 backgrounds, 0 corrupt: 100%|██████████| 375/375 [00:00<?, ?it/s]


optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to runs/detect/train74
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

       4/10         0G      2.846      1.754      1.135         22        416: 100%|██████████| 71/71 [02:19<00:00,  1.96s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:19<00:00,  1.65s/it]
  0%|          | 0/71 [00:00<?, ?it/s]

                   all        375       1132     0.0831     0.0813     0.0204    0.00504

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/10         0G      2.722      1.638      1.097         30        416: 100%|██████████| 71/71 [02:17<00:00,  1.94s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:19<00:00,  1.63s/it]
  0%|          | 0/71 [00:00<?, ?it/s]

                   all        375       1132      0.172      0.117     0.0471      0.013

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/10         0G      2.356      1.418      1.089         49        416:  17%|█▋        | 12/71 [00:23<01:59,  2.02s/it]

## Load the best model
Load the best.pt file saved from the Hyperparameter Tuning process

In [24]:
# Load the best model 
best_model = YOLO("/Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/runs/detect/train61/weights/best.pt")

Validate the best model

In [29]:
# Evaluate on test set defined in the YAML file
metrics = best_model.val(data="wisard_dataset_rgb.yaml", split="test")

Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 CPU (Apple M4 Pro)


val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/rgb/labels.cache... 142 images, 7 backgrounds, 0 corrupt: 100%|██████████| 142/142 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:16<00:00,  1.78s/it]


                   all        142        243       0.11      0.112     0.0277    0.00757
Speed: 0.2ms preprocess, 78.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val4


Use the model for inference

In [37]:
best_model.predict(
    source="datasets/WiSARD_dataset/registered/test/rgb/images",
    save=True,
    conf=0.25
)


image 1/142 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/rgb/images/210417_MtErie_Enterprise_VIS_0007_00000031.jpeg: 384x640 1 person, 32.6ms
image 2/142 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/rgb/images/210417_MtErie_Enterprise_VIS_0007_00000032.jpeg: 384x640 (no detections), 24.8ms
image 3/142 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/rgb/images/210417_MtErie_Enterprise_VIS_0007_00000033.jpeg: 384x640 (no detections), 38.1ms
image 4/142 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/rgb/images/210417_MtErie_Enterprise_VIS_0007_00000034.jpeg: 384x640 (no detections), 27.0ms
image 5/142 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/rgb/images/210417_MtErie_Enterpri

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person'}
 obb: None
 orig_img: array([[[ 75,  60,  58],
         [ 75,  60,  58],
         [ 74,  59,  57],
         ...,
         [ 71,  96, 106],
         [ 68,  93, 103],
         [ 63,  88,  98]],
 
        [[ 74,  59,  57],
         [ 74,  59,  57],
         [ 74,  59,  57],
         ...,
         [ 65,  90, 100],
         [ 59,  84,  94],
         [ 44,  69,  79]],
 
        [[ 73,  58,  56],
         [ 73,  58,  56],
         [ 72,  57,  55],
         ...,
         [ 59,  84,  94],
         [ 57,  82,  92],
         [ 48,  73,  83]],
 
        ...,
 
        [[ 22,  21,  23],
         [ 22,  21,  23],
         [ 22,  21,  23],
         ...,
         [160, 192, 211],
         [169, 199, 216],
         [171, 201, 218]],
 
        [[ 22,  21,  23],
         [ 22,  21,  23],
         [ 22,  21,  23],
         ...,
         [164, 197

# VOLOv8

## Hyperparameter Tuning

In [38]:
# Define search space
search_space = {
    "lr0": (1e-5, 1e-1),
    "batch": [5, 10, 20]
}

# Tune hyperparameters on WISARD DATASET
modelv8.tune(
    data="wisard_dataset_rgb.yaml",
    epochs=5,
    iterations=3,
    optimizer="Adam",
    space=search_space,
    device="cpu"
)

Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune12'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/3 with hyperparameters: {'lr0': 0.01, 'batch': 10}


/Users/ikmalbasirun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Transferred 355/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.121 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 CPU (Apple M4 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=wisard_dataset_rgb.yaml, epochs=5, time=None, patience=100, batch=10, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train67, exist_ok=False, pretrained=yolov8n.pt, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fals

train: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/rgb/labels.cache... 453 images, 11 backgrounds, 0 corrupt: 100%|██████████| 453/453 [00:00<?, ?it/s]
val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/labels.cache... 113 images, 1 backgrounds, 0 corrupt: 100%|██████████| 113/113 [00:00<?, ?it/s]


optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train67
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        1/5         0G      2.402      2.175      1.365         10        640: 100%|██████████| 46/46 [01:18<00:00,  1.70s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:11<00:00,  1.91s/it]
  0%|          | 0/46 [00:00<?, ?it/s]

                   all        113        200      0.764      0.065     0.0889     0.0226

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/5         0G       2.56      1.716      1.407         53        640:  72%|███████▏  | 33/46 [00:57<00:22,  1.77s/it]

KeyboardInterrupt: 

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
